In [35]:
import geopandas as gpd
import os
import pandas as pd
import requests
from io import BytesIO

In [ ]:
DEST_DATA="../data/"
URL_SECTIONS = "https://gis.comune.trento.it/dbexport?db=base&sc=demografici&ly=civici_elettorali&fr=shp"
URL_PREFERENCES_CANDIDATES = "https://www.comune.trento.it/content/download/1582227/15012199/file/Preferenze.csv"
URL_PREFERENCES_GROUPS = "https://www.comune.trento.it/content/download/1582230/15012211/file/Voti_Lista.csv"
URL_PREFERENCES_MAYORS = "https://www.comune.trento.it/content/download/1582233/15012223/file/Voti_Sindaco.csv"
URL_DISTRICTS = "https://gis.comune.trento.it/dbexport?db=base&sc=confini&ly=circoscrizioni&fr=shp"
URL_TURNOUT = "https://www.comune.trento.it/content/download/1582223/15012185/file/Affluenza.csv"
if os.path.exists(DEST_DATA) == False:
   os.mkdir(DEST_DATA) 

,id_district,district,id_section,Ora,Iscritte femmine,Iscritti maschi,max_voters,Votanti femmine,Votanti maschi,effective_voters
0,1,Gardolo,84,Ore 22,639,637,1276,237,260,497
1,1,Gardolo,85,Ore 22,458,464,922,229,243,472
2,1,Gardolo,86,Ore 22,447,428,875,239,248,487
3,1,Gardolo,87,Ore 22,568,531,1099,291,256,547
4,1,Gardolo,88,Ore 22,585,561,1146,259,237,496
...,...,...,...,...,...,...,...,...,...,...
93,12,Centro storico - Piedicastello,13,Ore 22,532,490,1022,301,262,563
94,12,Centro storico - Piedicastello,14,Ore 22,503,468,971,216,185,401
95,12,Centro storico - Piedicastello,15,Ore 22,478,483,961,239,246,485
96,12,Centro storico - Piedicastello,16,Ore 22,445,428,873,220,206,426


In [21]:
columns = {
    "Cod. circoscrizione":"id_district",
    "Circoscrizione":"district",
    "Sezione":"id_section",
    "Iscritti totale":"max_voters",
    "Votanti totale":"effective_voters"
}
turnout.rename(columns=columns,inplace=True)

In [5]:
turnout = turnout[['id_district','district','id_section','max_voters','effective_voters']]

In [6]:
turnout['turnout'] = (turnout["effective_voters"] / turnout["max_voters"] * 100).round(0)

In [10]:
turnout = turnout.sort_values(by='turnout', ascending=False)

In [ ]:
turnout[turnout.turnout > 50.0]

,id_district,district,id_section,max_voters,effective_voters,turnout
64,10,Oltrefersina,48,0,7,inf
26,6,Argentario,64,802,521,65.0
50,10,Oltrefersina,34,884,549,62.0
52,10,Oltrefersina,36,854,523,61.0
27,6,Argentario,65,1000,611,61.0
21,5,Ravina - Romagnano,73,902,548,61.0
70,11,S. Giuseppe S. Chiara,21,982,590,60.0
55,10,Oltrefersina,39,905,545,60.0
15,3,Bondone,78,1051,632,60.0
67,11,S. Giuseppe S. Chiara,18,904,529,59.0


In [ ]:
districts_affluence = turnout.groupby("district", as_index=False).agg({
    "effective_voters": "sum",
    "max_voters": "sum"
})


In [ ]:
districts_affluence["turnout_percent"] = (
    districts_affluence["effective_voters"] / districts_affluence["max_voters"] * 100
).round(2)

In [ ]:
districts_affluence

,district,effective_voters,max_voters,turnout_percent
0,Argentario,6126,10690,57.31
1,Bondone,2655,4981,53.30
2,Centro storico - Piedicastello,7859,18793,41.82
3,Gardolo,5154,10850,47.50
4,Mattarello,2828,6980,40.52
5,Meano,2379,4629,51.39
6,Oltrefersina,8418,15530,54.20
7,Povo,2762,4938,55.93
8,Ravina - Romagnano,2325,5298,43.88
9,S. Giuseppe S. Chiara,7543,14219,53.05


DOWNLOAD

In [ ]:
sections = None
file_sections = DEST_DATA + "sezioni_elettorali.shp"
if os.path.exists(file_sections):
    sections = gpd.read_file(file_sections)
else:
    sections = gpd.read_file(URL_SECTIONS)
    sections.to_file(file_sections)
districts = None
file_districts = "circoscrizioni.shp"
if os.path.exists(file_districts):
    districts = gpd.read_file(file_districts)
else:
    districts = gpd.read_file(URL_DISTRICTS)
    districts.to_file(file_districts)    

In [ ]:
preferences_candidates = None
file_preferences_candidates = DEST_DATA + "Preferenze.csv"
if os.path.exists(file_preferences_candidates):
    preferences_candidates = pd.read_csv(file_preferences_candidates,sep=";",encoding="ISO-8859-15")
else:
    preferences_candidates = pd.read_csv(URL_PREFERENCES_CANDIDATES,sep=";",encoding="ISO-8859-15")
    preferences_candidates.to_csv(file_preferences_candidates,sep=";",encoding="ISO-8859-15",index=False)

preferences_groups = None
file_preferences_groups = DEST_DATA + "Voti_Lista.csv"
if os.path.exists(file_preferences_groups):
    preferences_groups = pd.read_csv(file_preferences_groups,sep=";",encoding="ISO-8859-15")
else:
    preferences_groups = pd.read_csv(URL_PREFERENCES_GROUPS,sep=";",encoding="ISO-8859-15")
    preferences_groups.to_csv(file_preferences_groups,sep=";",encoding="ISO-8859-15",index=False)
    
preferences_mayors = None
file_preferences_mayors = DEST_DATA + "Voti_Sindaco.csv"
if os.path.exists(file_preferences_mayors):
    preferences_mayors = pd.read_csv(file_preferences_mayors,sep=";",encoding="ISO-8859-15")
else:
    preferences_mayors = pd.read_csv(URL_PREFERENCES_MAYORS,sep=";",encoding="ISO-8859-15")
    preferences_mayors.to_csv(file_preferences_mayors,sep=";",encoding="ISO-8859-15",index=False)


In [ ]:
columns_districts = {
    'numero_cir':'id_district',
    'nome':'district'}
districts.rename(columns=columns_districts,inplace=True)
districts = districts[['id_district','district','geometry']].to_crs(epsg=4326)
columns_sections =  {
    "sez_elett": "id_section",
    "sede":"station",
    "desvia":"streetname",
    "civico_alf":"housenumber"
}
sections.rename(columns=columns_sections,inplace=True)
sections = sections.to_crs(epsg=4326)
sections = gpd.sjoin(sections, districts, how='left', predicate='within')
stations = sections[['id_section','station']].drop_duplicates().reset_index()[['id_section','station']]
sections = sections[['id_district','district','id_section','station','streetname','housenumber','geometry']].to_crs(epsg=4326)
stations = sections[['id_section','station']].drop_duplicates().reset_index()[['id_section','station']]

In [ ]:
def createIdCandidate(group,positiongroup):
    group = str(group).zfill(2)
    positiongroup = str(positiongroup).zfill(2)
    return (group+positiongroup)
preferences_candidates['id_candidate'] = preferences_candidates.apply(
    lambda row: createIdCandidate(row['Progressivo lista'], row['Progr. candidato']),
    axis=1
)

In [ ]:
preferences_columns = {
    'Sezione':'id_section',
    'Cod. circoscrizione':'id_district',
    'Circoscrizione':'district',
    'Nome Lista':'group',
    'Cognome':'lastname',
    'Nome':'name',
    'Nome Detto':'nickname',
    'Voti':'votes'
}
preferences_candidates.rename(columns=preferences_columns,inplace=True)

preferences_columns = {
    'Sezione':'id_section',
    'Cod. circoscrizione':'id_district',
    'Circoscrizione':'district',
    'Lista':'group',
    'Sindaco':'supportedmayor',
    'Voti':'votes'
}
preferences_groups.rename(columns=preferences_columns,inplace=True)
preferences_groups = preferences_groups.dropna(subset=['supportedmayor'])


In [ ]:
preferences_groups['id_section'] = preferences_groups['id_section'].fillna(-1)
preferences_groups['id_section'] = preferences_groups['id_section'].astype(int)
preferences_groups['id_district'] = preferences_groups['id_district'].fillna(-1)
preferences_groups['id_district'] = preferences_groups['id_district'].astype(int)

In [ ]:
preferences_candidates = preferences_candidates.merge(
    preferences_groups[['group', 'supportedmayor']].drop_duplicates(),
    on='group',
    how='left'
)
preferences_columns = {
    "Sezione":"id_section",
    "Cod. circoscrizione":"id_district",
    "Circoscrizione":"district",
    "Schede Bianche":"blanks",
    "Schede nulle o contenenti solo voti nulli":"canceled",
    "Voti contestati e non attribuiti":"disputed",
    "Cognome":"lastname",
    "Nome":"name",
    "Voti":"votes"
}
preferences_mayors.rename(columns=preferences_columns,inplace=True)

Testing

In [ ]:
from fuzzywuzzy import process

def get_section_code(address, housenumber):
    # Estrai il nome della via dall'indirizzo
    streetname = address.upper()
    
    # Trova la via più simile in sections
    best_match = process.extractOne(streetname, sections['streetname'])
    
    if best_match:
        matched_street = best_match[0]
        # Filtra per via e numero civico
        result = sections[(sections['streetname'] == matched_street) & (sections['housenumber'] == housenumber)]
        if not result.empty:
            return result.iloc[0]['id_section']
    return None

def get_votes_by_candidate_and_section(first_name, last_name, section_id):
    # Filtra il dataframe preferences_candidates per nome, cognome e id_section
    filtered = preferences_candidates[
        (preferences_candidates['name'] == first_name.upper()) &
        (preferences_candidates['lastname'] == last_name.upper()) &
        (preferences_candidates['id_section'] == section_id)
    ]
    
    # Restituisci il numero di voti se esiste una corrispondenza
    if not filtered.empty:
        return filtered.iloc[0]['votes']
    return 0


In [ ]:
first_name = "Maurizio"
last_name = "Napolitano"
address = "Herrsching"
housenumber = "8"
section_id = get_section_code(address, housenumber)


In [ ]:
votes = get_votes_by_candidate_and_section(first_name, last_name, section_id)
print(f"Numero di voti per {first_name} {last_name} nella sezione {section_id}: {votes}")

Numero di voti per Maurizio Napolitano nella sezione 74: 2


In [ ]:
preferences_candidates[(preferences_candidates.lastname == "NAPOLITANO") &
                       (preferences_candidates.id_district == 5)]

,id_section,id_district,district,Progressivo lista,group,Progr. candidato,lastname,name,nickname,votes,id_candidate,supportedmayor
41482,72,5,RAVINA-ROMAGNANO,10,INSIEME PER TRENTO,29,NAPOLITANO,MAURIZIO,NaN,1,1029,IANESELLI FRANCO
42061,73,5,RAVINA-ROMAGNANO,10,INSIEME PER TRENTO,29,NAPOLITANO,MAURIZIO,NaN,0,1029,IANESELLI FRANCO
42640,74,5,RAVINA-ROMAGNANO,10,INSIEME PER TRENTO,29,NAPOLITANO,MAURIZIO,NaN,2,1029,IANESELLI FRANCO
43219,75,5,RAVINA-ROMAGNANO,10,INSIEME PER TRENTO,29,NAPOLITANO,MAURIZIO,NaN,0,1029,IANESELLI FRANCO
54799,95,5,RAVINA-ROMAGNANO,10,INSIEME PER TRENTO,29,NAPOLITANO,MAURIZIO,NaN,0,1029,IANESELLI FRANCO


Analisys